In [1]:
import pyspark
from pyspark.sql import SparkSession

In [20]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()

In [3]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

--2022-02-23 21:41:54--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
Распознаётся nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 54.231.131.33
Подключение к nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|54.231.131.33|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа... 200 OK
Длина: 752335705 (717M) [text/csv]
Сохранение в каталог: ««fhvhv_tripdata_2021-01.csv»».

fhvhv_tripdata_2021 100%[===================>] 717,48M  11,9MB/s    за 71s     

2022-02-23 21:43:22 (10,0 MB/s) - «fhvhv_tripdata_2021-01.csv» сохранён [752335705/752335705]



In [3]:
!wc -l fhvhv_tripdata_2021-01.csv

11908469 fhvhv_tripdata_2021-01.csv


In [21]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [22]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [23]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [14]:
!pip install pandas

     |████████████████████████████████| 11.7 MB 3.0 MB/s eta 0:00:01
  Using cached pytz-2021.3-py2.py3-none-any.whl (503 kB)
     |████████████████████████████████| 16.8 MB 30.5 MB/s eta 0:00:01


In [7]:
import pandas as pd

In [24]:
df_pandas = pd.read_csv('head.csv')

In [25]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [26]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [27]:
from pyspark.sql import types

In [28]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [29]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [14]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [30]:
df = df.repartition(24)

In [31]:
df.write.parquet('fhvhv/2021/01/', mode='overwrite')

22/02/27 13:02:47 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
22/02/27 13:02:47 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
22/02/27 13:02:47 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
22/02/27 13:02:47 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 69,09% for 11 writers
22/02/27 13:02:47 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 63,33% for 12 writers
22/02/27 13:02:49 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 69,09% for 11 writers
22/02/27 13:02:49 WARN MemoryManager: Total allocation exceeds 95,

In [32]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [34]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003') \
  .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-02 23:03:28|2021-01-02 23:10:18|          41|          42|
|2021-01-02 18:43:42|2021-01-02 19:02:09|          11|         178|
|2021-01-01 06:56:49|2021-01-01 07:07:03|          39|          89|
|2021-01-01 18:04:56|2021-01-01 18:13:30|          74|         247|
|2021-01-01 14:10:21|2021-01-01 14:20:31|          33|         255|
|2021-01-02 16:49:09|2021-01-02 17:04:46|          80|          49|
|2021-01-01 03:04:15|2021-01-01 03:17:31|         189|          17|
|2021-01-01 00:42:12|2021-01-01 01:08:54|          61|          76|
|2021-01-01 19:35:19|2021-01-01 19:56:54|         112|         188|
|2021-01-01 17:45:23|2021-01-01 17:57:04|          89|         188|
|2021-01-02 21:56:45|2021-01-02 22:06:10|          74|          42|
|2021-01-02 22:42:26|2021-01-02 22:54:10|       

In [35]:
from pyspark.sql import functions as F

In [ ]:
F.to_date()

In [37]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-02|  2021-01-02|         185|         159|
| 2021-01-01|  2021-01-01|         180|         258|
| 2021-01-02|  2021-01-02|         248|         185|
| 2021-01-02|  2021-01-02|         236|         148|
| 2021-01-03|  2021-01-03|          75|          41|
| 2021-01-01|  2021-01-01|         138|         198|
| 2021-01-01|  2021-01-01|         209|          45|
| 2021-01-02|  2021-01-02|          73|         129|
| 2021-01-01|  2021-01-01|          39|          35|
| 2021-01-02|  2021-01-02|          18|         169|
| 2021-01-01|  2021-01-01|         114|          79|
| 2021-01-02|  2021-01-02|         216|         215|
| 2021-01-02|  2021-01-02|           7|         130|
| 2021-01-03|  2021-01-03|         242|          41|
| 2021-01-02|  2021-01-02|         191|         218|
| 2021-01-02|  2021-01-02|         246|       

In [38]:
df.count()

11908468